In [1]:
import numpy as np

In [19]:
a = np.array([[-1,-1], [0,0.1], [1, 1.6]])

In [20]:
a.shape

(3, 2)

In [21]:
outered = np.subtract.outer(a,a)

In [30]:
a1, a2 = list(a.T)
np.subtract.outer(a1, a1)

array([[ 0., -1., -2.],
       [ 1.,  0., -1.],
       [ 2.,  1.,  0.]])

In [97]:
np.outer?

In [148]:
def dists(rs, L):
    """
    Returns the difference vectors between all pairs of points in rs, given a periodic box of size L.
    
    Parameters
    ----------
    rs : (N,d) array_like
         Input vectors
    L : (d,) array_like, or scalar
    Size of the box
    
    Returns
    -------
    out : (N,N,d) ndarray
    Where out[i,j,:] is the difference vector between rs[i,:] and rs[j,:]
    """
    diffs = np.array([np.subtract.outer(rd, rd) for rd in rs.T]).T
    return np.remainder(diffs + L/2., L) - L/2.

In [99]:
d = dists(a, 1.0)
print(d.shape)
d

(3, 3, 2)


array([[[ 0. ,  0. ],
        [ 0. ,  0.1],
        [ 0. , -0.4]],

       [[ 0. , -0.1],
        [ 0. ,  0. ],
        [ 0. , -0.5]],

       [[ 0. ,  0.4],
        [ 0. , -0.5],
        [ 0. ,  0. ]]])

In [145]:
N, L = 800, 2
d = np.random.uniform(high=L, size=(N,2))
dfs = dists(d, L)
drs = np.sqrt(np.sum(dfs**2, axis=-1))
np.sum(drs[np.triu_indices_from(drs)] < 1) / (N*(N+1)//2)

0.78602996254681645

In [67]:
drs[np.triu_indices_from(drs)].shape

(55,)

In [103]:
np.zeros(10)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [144]:
%%timeit -n 1 -r 1
N = 200
d = 3
L = 2.0
Nruns = 500

in_disk = np.zeros(Nruns)
for i in range(Nruns):
    rs = np.random.uniform(size=(N,d), high=(L*2))
    diffs = dists(rs, L)
    drs = np.sqrt(np.sum(diffs**2, axis=-1))
    #in_disk[i] = np.sum(drs[np.triu_indices_from(drs)] < 1)
    in_disk[i] = (np.sum(drs < 1) - N) / 2

In [149]:
def dists2(rs, L):
    """
    Returns the difference vectors between all pairs of points in rs, given a periodic box of size L.
    
    Parameters
    ----------
    rs : (N,d) array_like
         Input vectors
    L : (d,) array_like, or scalar
    Size of the box
    
    Returns
    -------
    out : (N,N,d) ndarray
    Where out[i,j,:] is the difference vector between rs[i,:] and rs[j,:]
    """
    N,d = np.shape(rs)
    diffs = np.zeros((N,N,d))
    for i,rd in enumerate(rs.T):
        np.subtract.outer(rd, rd, out=diffs[:,:,i])
    return np.remainder(diffs + L/2., L) - L/2.

In [154]:
%%timeit rs = np.random.uniform(size=(100,3), high=(4.0))
diffs = dists(rs, 2.0)

1000 loops, best of 3: 2.63 ms per loop


In [143]:
diffs

array([[[ 0.        ,  0.        ,  0.        ],
        [ 0.67767265,  0.27251136,  0.69346879],
        [ 0.08837079,  0.98017716, -0.7722809 ],
        ..., 
        [-0.3291995 , -0.19736187, -0.57391698],
        [ 0.24014932, -0.94804328, -0.53381377],
        [ 0.36993133,  0.78150857, -0.01996707]],

       [[-0.67767265, -0.27251136, -0.69346879],
        [ 0.        ,  0.        ,  0.        ],
        [-0.58930186,  0.7076658 ,  0.53425031],
        ..., 
        [ 0.99312785, -0.46987323,  0.73261422],
        [-0.43752333,  0.77944536,  0.77271744],
        [-0.30774133,  0.50899721, -0.71343586]],

       [[-0.08837079, -0.98017716,  0.7722809 ],
        [ 0.58930186, -0.7076658 , -0.53425031],
        [ 0.        ,  0.        ,  0.        ],
        ..., 
        [-0.41757029,  0.82246098,  0.19836392],
        [ 0.15177853,  0.07177956,  0.23846713],
        [ 0.28156053, -0.19866859,  0.75231383]],

       ..., 
       [[ 0.3291995 ,  0.19736187,  0.57391698],
        